<a href="https://colab.research.google.com/github/flamemeister/QazQa/blob/main/Train_kazT5_base_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import unicodedata

import torch
from torch.utils.data import DataLoader

from transformers import T5Tokenizer, T5TokenizerFast
from transformers import T5ForConditionalGeneration
from transformers import AdamW
from transformers import BertForQuestionAnswering

from tqdm import tqdm
import sentencepiece

from sklearn.model_selection import train_test_split 
from datetime import datetime

max_text_length   = 256

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/НИРС/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_data(path):
    df = pd.read_csv(path, encoding='utf-16')
    df = df.drop('Index', axis=1)
    data = df.values.tolist()
    data = [item[0] for item in data]
    return data

def to_utf8(text):
    str(text).encode(encoding = 'UTF-16', errors = 'strict')
    return text

In [ ]:
train_size = 30000

contexts  = load_data(path + 'Kaz_SQuAD/train_contexts_pdcsv.csv')[0:train_size]
questions = load_data(path + 'Kaz_SQuAD/train_questions_pdcsv.csv')[0:train_size]
answers   = load_data(path + 'Kaz_SQuAD/train_answers_pdcsv.csv')[0:train_size]

In [ ]:
c_train, c_val, q_train, q_val, a_train, a_val = train_test_split(
    contexts, questions, answers, 
    test_size=0.2,
    random_state=73)

In [1]:
%%capture
model     = T5ForConditionalGeneration.from_pretrained(path + 'KazNLP/Tokenizers/t5-kaz-base')
tokenizer = T5Tokenizer.from_pretrained(path+ 'KazNLP/Tokenizers/t5-kaz-base')

NameError: ignored

In [ ]:
def add_token_positions(encodings, answers):
    tokenized = tokenizer(answers, truncation=True, padding=True)
    encodings.update({'target_ids': tokenized['input_ids'], 'target_attention_mask': tokenized['attention_mask']})

train_encodings = tokenizer(c_train, q_train, max_length=max_text_length, truncation=True, padding=True)
val_encodings   = tokenizer(c_val, q_val, max_length=max_text_length, truncation=True, padding=True)

add_token_positions(train_encodings, a_train)
add_token_positions(val_encodings, a_val)

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        print(encodings.keys())
        
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset   = SquadDataset(val_encodings)
train_loader  = DataLoader(train_dataset, batch_size=5, shuffle=True)

In [ ]:
model.train()

def save_model():
    now = datetime.now()
    date_time = now.strftime(" %m %d %Y %H %M %S")
    torch.save(model.state_dict(), "nlpModel"+date_time+".pt")

In [ ]:
cuda = torch.cuda.is_available()
device = torch.device('cuda') if cuda else torch.device('cpu')
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=1e-4)

In [ ]:
for epoch in range(5):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()

        # >
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=target_ids,
                        decoder_attention_mask=target_attention_mask)
        # >
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


In [ ]:
cuda = torch.cuda.is_available()
device = torch.device('cuda') if cuda else torch.device('cpu')
model.to(device)
model.eval()

In [ ]:
def test(context, question):
    input = tokenizer(context,
                      question,
                      max_length=max_text_length,
                      truncation=True,
                      padding=True)
    
    with torch.no_grad():
        input_ids = torch.tensor(input['input_ids']).to(device)
        attention_mask = torch.tensor(input['attention_mask']).to(device)
        out = model.generate(input_ids,
                             attention_mask=attention_mask,
                             max_length=max_text_length,
                             early_stopping=False)
        return([tokenizer.decode(ids, skip_special_tokens=True) for ids in out[0]])

In [ ]:
story = ["Алматыда оқушылар мен студенттер онлайн оқиды."]
ques = ["Алматыда кім оқиды?"]

print(test(story, ques))